# Time-MoE

https://github.com/Time-MoE/Time-MoE

In [ ]:

!git clone https://github.com/Time-MoE/Time-MoE.git

In [ ]:
%cd ./Time-MoE

In [ ]:
pip install -r requirements.txt

In [ ]:
pip install flash-attn==2.6.3

Forecasting Sample (from https://github.com/Time-MoE/Time-MoE)

In [ ]:
import numpy as np
import torch
from transformers import AutoModelForCausalLM

# context_length = 12
# seqs = torch.randn(2, context_length)  # tensor shape is [batch_size, context_length]
# seqs = np.array([
#     1., 1.1, 1.2, 1.1, 1.0, 0.9, 0.8, 0.7, 0.7, 0.8, 0.9, 1., 1.1, 1.2
# ])
seqs = [
    1., 1.1, 1.2, 1.1, 1.0, 0.9, 0.8, 0.7, 0.7, 0.8, 0.9, 1., 1.1, 1.2
]
prediction_length = 2
context_length = len(seqs) - prediction_length 
seqs = torch.Tensor([seqs[0:context_length]])

model = AutoModelForCausalLM.from_pretrained(
    'Maple728/TimeMoE-50M',
    device_map="cpu",  # use "cpu" for CPU inference, and "cuda" for GPU inference.
    trust_remote_code=True,
)

# use it when the flash-attn is available
# model = AutoModelForCausalLM.from_pretrained('Maple728/TimeMoE-50M', device_map="auto", attn_implementation='flash_attention_2', trust_remote_code=True)

# normalize seqs
mean, std = seqs.mean(dim=-1, keepdim=True), seqs.std(dim=-1, keepdim=True)
normed_seqs = (seqs - mean) / std

# forecast
# prediction_length = 6
output = model.generate(normed_seqs, max_new_tokens=prediction_length)  # shape is [batch_size, 12 + 6]
normed_predictions = output[:, -prediction_length:]  # shape is [batch_size, 6]

# inverse normalize
predictions = normed_predictions * std + mean

print(predictions)

In [ ]:
!python run_eval.py -d ../dataset/ETT-small/ETTh1.csv -p 96